<a href="https://colab.research.google.com/github/brayanricardo13/Analisis-Econometrico/blob/main/PRECIA_VALORACI%C3%93N_RENTA_FIJA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SX - MX download**

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300

Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.Dg2BOdtQWO/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.aP6LPg6knU/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.JMb7dIyPRb/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 ht

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

In [3]:
import os
import time
import warnings
import numpy as np
import pandas as pd

In [4]:
from IPython.display import Image
from bs4 import BeautifulSoup
from pandas.tseries.offsets import MonthEnd

In [5]:
warnings.filterwarnings("ignore")


In [6]:
def web_driver():
  chrome_options = webdriver.ChromeOptions()

  chrome_options.add_argument('--verbose')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--disable-gpu')
  chrome_options.add_argument('--window-size=1420,1080')
  chrome_options.add_argument('--disable-dev-shm-usage')

  driver = webdriver.Chrome(options=chrome_options)

  return driver



In [7]:
def month_days_range(year, month):

    base_date = pd.Timestamp(year, month, 1)

    first_day = base_date.day
    last_day = (base_date + MonthEnd()).day

    return range(first_day,last_day + 1)


In [8]:
def date_range(year, month):

    initial_date = pd.Timestamp(year, month, 1)
    final_date = (initial_date + MonthEnd())

    first_day = initial_date.day
    last_day = final_date.day

    days_range_ = range(first_day,last_day + 1)
    date_range_ = pd.date_range(initial_date,final_date)

    return date_range_, days_range_

In [9]:
def download_sx(driver,year,month,day):

  MONTHS = {1:'Ene',2:'Feb',3:'Mar',4:'Abr',5:'May',6:'Jun',7:'Jul',8:'Ago',9:'Sep',10:'Oct',11:'Nov',12:'Dic'}
  month = MONTHS[month]

  select_element_year = driver.find_element(by = 'xpath',value = '//form[@id = "form" and @method = "post"]//div[@class = "ui-fieldset-content"]//span//div/select[@class = "ui-datepicker-year"]')
  select_year = Select(select_element_year)
  select_year.select_by_visible_text(str(year))

  time.sleep(3)
  select_element_month = driver.find_element(by = 'xpath',value = '//form[@id = "form" and @method = "post"]//div[@class = "ui-fieldset-content"]//span//div/select[@class = "ui-datepicker-month"]')
  select_month = Select(select_element_month)
  select_month.select_by_visible_text(month)

  time.sleep(3)
  day = driver.find_element(by = 'xpath',value = '//form[@id = "form" and @method = "post"]//div[@class = "ui-fieldset-content"]//span//table/tbody/tr/td[@data-handler = "selectDay"]/a[text() = {0}]'.format(day))

  time.sleep(3)
  day.click()

  time.sleep(2)
  sx = driver.find_element(by = 'xpath',value = '//form[@id = "form" and @method = "post"]//div[@class = "ui-fieldset-content"]/div[@id = "form:TablaValoracionRentaFija"]/div[@class="ui-datatable-tablewrapper"]//tbody/tr[contains(@data-rk,"SX")]/td[last()]//a')

  time.sleep(3)
  sx.click()

In [10]:
def reload(driver):

    driver.get(URLS['renta_fija'])

    renta_fija = driver.find_element(by = 'id',value = 'arin')
    time.sleep(3)
    renta_fija.click()

    iframe = driver.find_element(by = 'id',value =  'BranderFrame')
    driver.switch_to.frame(iframe)

In [11]:
def download_mx_loop(year,month,day_range,today):

    range_ = np.array(day_range)
    indexes = list()

    for index, day in enumerate(day_range):
        date = pd.Timestamp(year,month,day)

        if not(date < today):
          break

        try:
            download_mx(driver,year,month,day)
            print("MX del {0} descargado.".format(date.strftime('%Y-%m-%d')))
            indexes.append(index)
            time.sleep(3)

        except:
            print("\tRevisar: MX del {0} no descargado.".format(date.strftime('%Y-%m-%d')))
            time.sleep(3)

            return range(range_[range_ >= (day)].min(),range_[range_ > (day - 1)].max() + 1)
            break

    return None

In [12]:
def download_mx(driver,year,month,day):

  MONTHS = {1:'Ene',2:'Feb',3:'Mar',4:'Abr',5:'May',6:'Jun',7:'Jul',8:'Ago',9:'Sep',10:'Oct',11:'Nov',12:'Dic'}
  month = MONTHS[month]

  select_element_year = driver.find_element(by = 'xpath',value = '//form[@id = "form" and @method = "post"]//div[@class = "ui-fieldset-content"]//span//div/select[@class = "ui-datepicker-year"]')
  select_year = Select(select_element_year)
  select_year.select_by_visible_text(str(year))

  time.sleep(3)
  select_element_month = driver.find_element(by = 'xpath',value = '//form[@id = "form" and @method = "post"]//div[@class = "ui-fieldset-content"]//span//div/select[@class = "ui-datepicker-month"]')
  select_month = Select(select_element_month)
  select_month.select_by_visible_text(month)

  time.sleep(3)
  day = driver.find_element(by = 'xpath',value = '//form[@id = "form" and @method = "post"]//div[@class = "ui-fieldset-content"]//span//table/tbody/tr/td[@data-handler = "selectDay"]/a[text() = {0}]'.format(day))

  time.sleep(3)
  day.click()

  time.sleep(2)
  mx = driver.find_element(by = 'xpath',value = '//form[@id = "form" and @method = "post"]//div[@class = "ui-datatable-tablewrapper"]//tbody/tr//label[contains(text(),"MX") and not(contains(text(),"_"))]/parent::td/following-sibling::td//a')

  time.sleep(3)
  mx.click()

In [13]:
def sx_download(driver:webdriver,year:int,month:int,day:int = None):

  year_range = years_range(driver)
  month_range = range(1,12 + 1)
  date_range_, day_range = date_range(year, month)

  condition1 = year in year_range
  condition2 = month in month_range
  condition3 = day in day_range

  conditions1 = np.all([condition1,condition2,condition3])
  conditions2 = np.all([condition1,condition2])

  if conditions1:
    date = pd.Timestamp(year,month,day)

    try:
      download_sx(driver,year,month,day)
      print("SX del {0} descargado.".format(date.strftime('%Y-%m-%d')))
      time.sleep(2)
    except:
      print("\tRevisar: SX del {0} no descargado.".format(date.strftime('%Y-%m-%d')))
      time.sleep(2)

  elif (day is None) and conditions2:

    today_date = pd.Timestamp.today(tz = 'America/Bogota')
    today = pd.Timestamp(today_date.year,today_date.month,today_date.day)

    for day in day_range:

      date = pd.Timestamp(year,month,day)

      if not(date < today):
        break

      try:
        download_sx(driver,year,month,day)
        print("SX del {0} descargado.".format(date.strftime('%Y-%m-%d')))
        time.sleep(3)
      except:
        print("\tRevisar: SX del {0} no descargado.".format(date.strftime('%Y-%m-%d')))
        time.sleep(3)



In [14]:
def mx_download(driver:webdriver,year:int,month:int,day:int = None):

  year_range = years_range(driver)
  month_range = range(1,12 + 1)
  date_range_, day_range = date_range(year, month)

  condition1 = year in year_range
  condition2 = month in month_range
  condition3 = day in day_range

  conditions1 = np.all([condition1,condition2,condition3])
  conditions2 = np.all([condition1,condition2])

  today_date = pd.Timestamp.today(tz = 'America/Bogota')
  today = pd.Timestamp(today_date.year,today_date.month,today_date.day)

  if conditions1:
    date = pd.Timestamp(year,month,day)

    try:
      download_mx(driver,year,month,day)
      print("MX del {0} descargado.".format(date.strftime('%Y-%m-%d')))
      time.sleep(2)
    except:
      print("\tRevisar: MX del {0} no descargado.".format(date.strftime('%Y-%m-%d')))
      time.sleep(2)

  elif (day is None) and conditions2:

      range_ = download_mx_loop(year,month,day_range,today)

      while not(range_ is None):
          range_ = download_mx_loop(year,month,range_,today)
          reload(driver)

      df_mx_files = files_dataframe(os.getcwd(),date_range_,'mx')
      loading_lacking_mx(df_mx_files,date_range_,driver)


In [15]:
def loading_lacking_mx(df_mx_files,date_range,driver):

    if len(df_mx_files.loc[df_mx_files['name'].isna()].index) == 0:

      min_date = df_mx_files.index.min().strftime('%Y-%m-%d')
      max_date = df_mx_files.index.max().strftime('%Y-%m-%d')

      print("\nTodos los MXs desde {0} hasta {1} han sido descargados.".format(min_date,max_date))

    else:

      print("\nLos siguientes MXs no han sido descargados correctamente:\n")

      for date in df_mx_files.loc[df_mx_files['name'].isna()].index:

        print("\tMX del {0}".format(date.strftime('%Y-%m-%d')))

      print("\nReanudando proceso.\n")

      for date in df_mx_files.loc[df_mx_files['name'].isna()].index:

        try:
          mx_download(driver,date.year,date.month,date.day)
        except:
          reload(driver)
          mx_download(driver,date.year,date.month,date.day)

      df_mx_files = files_dataframe(os.getcwd(),date_range,'mx')
      loading_lacking_mx(df_mx_files,date_range,driver)


In [16]:
### returns list of subdirectories and list of files within a directory

def list_dir(directory):

    directories, files = list(), list()

    for filename in os.listdir(directory):

        filepath = os.path.join(directory, filename)

        if os.path.isfile(filepath):
            files.append(filepath)
        elif os.path.isdir(filepath):
            directories.append(filepath)

    return directories, files

In [17]:
### returns list of all files within a directory

def list_files(directory):

    directories, files = list_dir(directory)

    while len(directories) > 0:

        directory = directories.pop(0)
        directories_, files_ = list_dir(directory)

        directories.extend(directories_)
        files.extend(files_)

    return files

In [18]:
### returns dataframe with date, name and filepath for the sx or mx in a given date range

def files_dataframe(directory,date_range,name):

    files = list_files(directory)
    files = [file for file in files if name.upper() in file]

    df_filepath = pd.DataFrame(columns = ['date','name','filepath'])

    for file in files:
        date = pd.to_datetime('20' + file[-6:-4] + '-' + file[-10:-8] + '-' + file[-8:-6])
        filepath = file

        name_ = 'df_{0}_'.format(name) + str(date.year) +str(date.month).zfill(2) + str(date.day).zfill(2)


        df_filepath = df_filepath.append({'date':date,
                                          'name':name_,
                                          'filepath':filepath},
                                          ignore_index=True)

    df_filepath.set_index(['date'],inplace = True)
    df_filepath.index.name = None

    df_mx = pd.DataFrame(index = date_range).pipe(
                lambda df:
                    df
                    .merge(df_filepath,
                           left_index = True,
                           right_index = True,
                           how = 'left')
                    .sort_index()
                )

    return df_mx

**Credenciales**

# **Credenciales**

In [19]:
CREDENTIALS = {
    'user':'diana.rodriguez@fiduciariacorficolombiana.com',
    'password':'dic$2020'
}

In [20]:

URLS = {
    'login':'https://www.precia.co/wp-login.php',
    'renta_fija':'https://www.precia.co/index.php/renta-fija/'
}

In [21]:
driver = web_driver()
driver.get(URLS['login'])

SessionNotCreatedException: ignored

In [ ]:
user.send_keys(CREDENTIALS['user'])
password.send_keys(CREDENTIALS['password'])

In [ ]:
time.sleep(3)
access.click()

#**SX**

In [ ]:
# Image(driver.get_screenshot_as_png())

In [ ]:
driver.get(URLS['renta_fija'])

In [ ]:
renta_fija = driver.find_element(by = 'id',value = 'arlo')
time.sleep(3)
renta_fija.click()

In [ ]:
iframe = driver.find_element(by = 'id',value =  'BranderFrame')
driver.switch_to.frame(iframe)

In [ ]:
sx_download(driver,2023,8,1)